# In this example we select a list of science window and submit it to ODA to obtain an image

In [1]:
# with no surprise, we do it for the Crab
source_name='Crab'

In [2]:
#Example of functions to select a science window list
import time

import astroquery.heasarc
from astropy.coordinates import SkyCoord
from astropy import units as u

Heasarc = astroquery.heasarc.Heasarc()



def get_scw_list(ra_obj, dec_obj,radius,start_date,end_date ):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'), 
            radius = f"{radius} deg",
            mission = 'intscw',                 
            time = start_date + " .. " + end_date,
            good_isgri = ">1000",
        )        

    R.sort('SCW_ID')

    return R['SCW_ID'], R['SCW_VER']


assert astroquery.__version__ >= '0.4.2.dev6611'

# it means it's our fork
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In [3]:
from astroquery.simbad import Simbad
from astropy import coordinates as coord

import json

import os
import shutil
import random

from astropy.io import fits

from scipy import stats
result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))
ra=source_coord.ra.deg
dec=source_coord.dec.deg
radius=8.

- We get science windows within 10 degrees from the Crab position 
- between the given dates expressed in the ISOT format

In [4]:
scwlist,version = get_scw_list(ra,dec, 10, '2020-01-01T00:00:00', '2020-06-01T00:00:00')

- We pick at random nscw = 10 science window from the list

In [5]:

nscw = 10
if len(scwlist) == 0:
    raise Exception('No science windows')

scw_pick = [(s+"."+v).strip() for s,v in zip(scwlist,version)]

if nscw >0 and len(scw_pick)>=nscw:
    random.seed(0)
    scw_pick = random.sample(scw_pick, nscw)
scw_pick = sorted(scw_pick)

In [6]:
scw_pick

['219400710010.001',
 '221000540010.001',
 '221200100010.001',
 '221400280010.001',
 '221400420010.001',
 '221400520010.001',
 '221500030010.001',
 '221500320010.001',
 '221500360010.001',
 '221500490010.001']

- Token for a given user 

In [7]:
token=''

In [8]:
#You can provide a valid token as explained in the 'Authentication' example or skip this cell
import getpass
token = getpass.getpass('Insert the token')

Insert the token········


## Let's get some logging

This is to help visualizing the progress.

- WANRING is the default level
- INFO writes some more information
- DEBUG is maily for developers and issue tracking


In [15]:
import logging
#default
#logging.getLogger().setLevel(logging.WARNING)
#slightly more verbose
logging.getLogger().setLevel(logging.DEBUG)
#all messages
#logging.getLogger().setLevel(logging.DEBUG)

logging.getLogger('oda_api').addHandler(logging.StreamHandler()) 

In [10]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url="https://www.astro.unige.ch/mmoda/dispatch-data", instrument="mock")

par_dict = {
"E1_keV": "28",
"E2_keV": "40",
"detection_threshold": "7",
"instrument": "isgri",
"osa_version": "OSA11.1",
"product": "isgri_image",
"product_type": "Real",
"scw_list": scw_pick,
"integral_data_rights": "all-private"    
}

if token != '':
    par_dict.update({'token': token})

data_collection = disp.get_product(**par_dict)

- waiting for remote response (since 2021-08-25 10:30:55), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c

... query status prepared => submitted
... assigned job id: e7846e3a8d133d9c
 | the job is working remotely, please wait status=submitted job_id=e7846e3a in 0 messages since 15 seconds (16/16); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-25 10:31:12), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c
 / the job is working remotely, please wait status=submitted job_id=e7846e3a in 0 messages since 26 seconds (13/16); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-25 10:31:23), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c
 - the job is working remotel

- waiting for remote response (since 2021-08-25 10:37:23), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c
 - the job is working remotely, please wait status=submitted job_id=e7846e3a in 0 messages since 400 seconds (16/39); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-25 10:37:36), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c
 \ the job is working remotely, please wait status=submitted job_id=e7846e3a in 0 messages since 413 seconds (16/39); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-25 10:37:50), please wait for https://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: N6ISIGQ8JTGEV49G job: e7846e3a8d133d9c
 | the job is working remotely, please wait status=submitted job_id=e7846e3a in 0 messages since 426 seco

In [11]:
data_collection.mosaic_image_0_mosaic.show()

------------------------------
name: mosaic_image
meta_data dict_keys(['product', 'instrument', 'src_name', 'query_parameters'])
number of data units 6
------------------------------
data uniti 0 ,name: PRIMARY
data uniti 1 ,name: GROUPING
data uniti 2 ,name: ISGR-MOSA-IMA
data uniti 3 ,name: ISGR-MOSA-IMA
data uniti 4 ,name: ISGR-MOSA-IMA
data uniti 5 ,name: ISGR-MOSA-IMA


In [12]:
data_collection.mosaic_image_0_mosaic.show_meta()

------------------------------
product : mosaic
instrument : isgri
src_name : 
query_parameters : [{"query_name": "isgri_image_query"}, {"product_name": "isgri_image_query"}, {"name": "detection_threshold", "units": "sigma", "value": "7"}, {"name": "image_scale_min", "units": None, "value": None}, {"name": "image_scale_max", "units": None, "value": None}]
------------------------------


In [16]:
from oda_api.plot_tools import OdaImage
#interactive
%matplotlib inline

im=OdaImage(data_collection.mosaic_image_0_mosaic)